In [2]:
import os
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
import numpy as np
import json

# Set paths to dataset
train_file = "train.json"
dev_file = "dev.json"
test_file = "Private_Test_ref.json"

# Define models to fine-tune
models = {
    "mBERT": "bert-base-multilingual-cased",
    "XLM-R": "xlm-roberta-base",
}

# Load evaluation metric
metric = load_metric("squad_v2")

# Function to preprocess the dataset
def prepare_train_features(examples, tokenizer, max_length=384, doc_stride=128):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

# Function to compute metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    start_logits, end_logits = predictions
    start_positions = labels["start_positions"]
    end_positions = labels["end_positions"]

    pred_start = np.argmax(start_logits, axis=1)
    pred_end = np.argmax(end_logits, axis=1)

    exact_match = np.mean((pred_start == start_positions) & (pred_end == end_positions))
    f1 = metric.compute(predictions={"pred_start": pred_start, "pred_end": pred_end}, references=labels)
    return {"exact_match": exact_match, "f1": f1}

# Load datasets
datasets = load_dataset("json", data_files={"train": train_file, "validation": dev_file, "test": test_file})

# Fine-tune each model
for model_name, model_checkpoint in models.items():
    print(f"Starting fine-tuning for {model_name}...")

    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

    # Tokenize datasets
    tokenized_datasets = datasets.map(
        lambda x: prepare_train_features(x, tokenizer),
        batched=True,
        remove_columns=datasets["train"].column_names,
    )

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f"./results/{model_name}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir=f"./logs/{model_name}",
        logging_steps=100,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    # Train the model
    trainer.train()

    # Evaluate on validation set
    validation_results = trainer.evaluate()
    print(f"Validation Results for {model_name}: {validation_results}")

    # Predict on test set
    predictions = trainer.predict(tokenized_datasets["test"])

    # Save predictions
    output_path = f"./results/{model_name}_predictions.json"
    with open(output_path, "w") as f:
        json.dump(predictions.metrics, f, indent=4)

    # Display results
    print(f"Test Results for {model_name}: {predictions.metrics}")

print("Fine-tuning and evaluation completed for all models.")


c:\Users\Lenovo\anaconda3\envs\preprocess\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


FileNotFoundError: Unable to find 'D:/STUDY_22521117/DS_3rd_year/semester_1st/[DS310]/LAB_DS310/LAB03\train.json'

In [ ]:
# Trainer
trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=tokenized_datasets["train"],
          eval_dataset=tokenized_datasets["validation"],
          tokenizer=tokenizer,
          compute_metrics=compute_metrics,
          )

# Fine-tune the model
trainer.train()

# Evaluate on the validation set
validation_results = trainer.evaluate()
print(f"Validation Results for {model_name}:")
print(validation_results)

# Make predictions on the test set
test_predictions = trainer.predict(tokenized_datasets["test"])

# Save predictions for later evaluation
test_predictions_dict = {
          "predictions": test_predictions.predictions,
          "metrics": test_predictions.metrics,
}
output_path = f"./results/{model_name}_test_predictions.json"
with open(output_path, "w") as f:
          import json
          json.dump(test_predictions_dict, f, indent=4)

# Print results
print(f"Test Results for {model_name}:")
print(test_predictions.metrics)

# Extract EM and F1 from metrics
em = test_predictions.metrics["exact_match"]
f1 = test_predictions.metrics["f1"]
print(f"EM: {em}, F1: {f1}")

print("Fine-tuning and evaluation completed for all models.")
